# --- `Day 15`: Chiton ---

In [33]:
import aocd
import re
import heapq
import operator
from collections import Counter, defaultdict, deque
from itertools import combinations
from functools import reduce, lru_cache

def prod(iterable):
    return reduce(operator.mul, iterable, 1)

def count(iterable, predicate = bool):
    return sum([1 for item in iterable if predicate(item)])

def first(iterable, default = None):
    return next(iter(iterable), default)

def lmap(func, *iterables):
    return list(map(func, *iterables))

def ints(s):
    return lmap(int, re.findall(r"-?\d+", s))

def words(s):
    return re.findall(r"[a-zA-Z]+", s)

def list_diff(x):
    return [b - a for a, b in zip(x, x[1:])]

def binary_to_int(lst):
    return int("".join(str(i) for i in lst), 2)

def get_column(lst, index):
    return [x[index] for x in lst]

In [3]:
def parse_line(line): 
    return lmap(int, line)
    
def parse_input(input):
    return list(map(parse_line, input.splitlines()))

In [4]:
final_input = parse_input(aocd.get_data(day=15, year=2021))
print(final_input[:5])

[[9, 1, 9, 7, 7, 9, 9, 9, 6, 7, 1, 4, 2, 9, 4, 9, 7, 1, 1, 9, 2, 4, 9, 1, 2, 5, 5, 9, 8, 5, 7, 2, 6, 6, 4, 2, 5, 9, 8, 9, 8, 9, 2, 8, 9, 5, 7, 9, 5, 3, 4, 9, 9, 8, 9, 9, 3, 5, 6, 7, 8, 8, 5, 2, 8, 5, 6, 4, 9, 1, 8, 6, 6, 5, 9, 7, 2, 4, 9, 8, 8, 3, 4, 5, 4, 3, 5, 5, 7, 2, 1, 8, 3, 8, 9, 9, 4, 6, 8, 9], [9, 4, 5, 4, 6, 9, 9, 4, 2, 8, 9, 5, 2, 9, 7, 7, 9, 1, 6, 2, 4, 8, 7, 9, 7, 6, 8, 7, 3, 7, 4, 1, 9, 9, 7, 9, 7, 5, 7, 9, 9, 3, 7, 1, 6, 6, 9, 5, 9, 9, 8, 5, 5, 7, 5, 9, 3, 4, 6, 8, 7, 7, 9, 9, 2, 8, 6, 2, 7, 8, 3, 9, 5, 7, 9, 9, 8, 3, 4, 5, 9, 9, 8, 2, 3, 7, 2, 5, 1, 9, 1, 7, 6, 8, 2, 6, 4, 2, 3, 9], [1, 4, 7, 6, 1, 2, 4, 5, 6, 7, 6, 7, 8, 9, 6, 2, 8, 9, 2, 9, 9, 8, 2, 4, 2, 1, 1, 4, 6, 5, 8, 3, 6, 8, 1, 6, 9, 9, 6, 9, 1, 9, 6, 9, 4, 9, 8, 9, 3, 7, 9, 9, 9, 6, 9, 9, 6, 8, 9, 3, 2, 4, 9, 2, 8, 7, 5, 8, 3, 5, 6, 6, 9, 9, 2, 8, 5, 3, 5, 8, 8, 8, 5, 1, 6, 5, 8, 5, 8, 8, 9, 5, 5, 7, 2, 9, 4, 8, 5, 7], [5, 7, 9, 1, 9, 1, 8, 2, 3, 9, 9, 9, 6, 6, 7, 8, 8, 3, 7, 7, 9, 8, 1, 8, 2, 1, 1, 9, 9, 9, 5,

In [5]:
test_input = parse_input('''\
1163751742
1381373672
2136511328
3694931569
7463417111
1319128137
1359912421
3125421639
1293138521
2311944581
''')

print(test_input)

[[1, 1, 6, 3, 7, 5, 1, 7, 4, 2], [1, 3, 8, 1, 3, 7, 3, 6, 7, 2], [2, 1, 3, 6, 5, 1, 1, 3, 2, 8], [3, 6, 9, 4, 9, 3, 1, 5, 6, 9], [7, 4, 6, 3, 4, 1, 7, 1, 1, 1], [1, 3, 1, 9, 1, 2, 8, 1, 3, 7], [1, 3, 5, 9, 9, 1, 2, 4, 2, 1], [3, 1, 2, 5, 4, 2, 1, 6, 3, 9], [1, 2, 9, 3, 1, 3, 8, 5, 2, 1], [2, 3, 1, 1, 9, 4, 4, 5, 8, 1]]


### Helpers

In [45]:
def printBoard(board, marked = set()):
    spacing = 3
    for r in range(min(20, len(board))):
        print(''.join(str.rjust(str(board[r][c]), spacing)
                if (r,c) not in marked else str.rjust('*', spacing) 
                for c in range(min(40, len(board[0])))))
    print("")
    
def get4Neighbors(input, r, c, w, h):
    #w,h = len(input[0]), len(input)
    offsets = [(-1,0), (0,1), (1,0), (0,-1)]
    result = []
    for (dr,dc) in offsets:
        rr = r + dr
        cc = c + dc
        if 0 <= rr < h and 0 <= cc < w:
            result.append((rr,cc))
    return result

## Solution 1

In [63]:
def solve_1(input):
    board = []
    for line in input:
        board.append([int(x) for x in line])
    w,h = len(board[0]), len(board)
    
    queue = [(0,0,0)]
    distances = [[1e9 for _ in range(w)] for _ in range(h)]
    distances[0][0] = 0
    visited = set()
    
    while queue:
        d,r,c = heapq.heappop(queue)
        visited.add((r, c))
        
        neighbors = get4Neighbors(board, r, c, w, h)
        for node in neighbors:
            if not node in visited:
                rr, cc = node
                cost = board[rr][cc] + distances[r][c]
                if cost < distances[rr][cc]:
                    distances[rr][cc] = cost
                else:
                    continue
                heapq.heappush(queue, (cost, rr, cc))
    #printBoard(distances)
    return distances[h - 1][w - 1]
    

solve_1(test_input)

40

In [64]:
f"Solution 1: {solve_1(final_input)}"

'Solution 1: 741'

## Solution 2

In [57]:
def solve_2(input):
    board = []
    for line in input:
        board.append([int(x) for x in line])
    w,h = len(board[0]), len(board)
    
    queue = [(0,0,0)]
    distances = [[1e9 for _ in range(w * 5)] for _ in range(h * 5)]
    distances[0][0] = 0
    visited = set()
    
    while queue:
        d,r,c = heapq.heappop(queue)
        visited.add((r, c))
        
        neighbors = get4Neighbors(board, r, c, w * 5, h * 5)
        for node in neighbors:
            if not node in visited:
                rr, cc = node
                value = board[rr % h][cc % w] + (rr // h) + (cc // w)
                while value > 9:
                    value -= 9
                    
                cost = value + distances[r][c]
                if cost < distances[rr][cc]:
                    distances[rr][cc] = cost
                else:
                    continue
                heapq.heappush(queue, (cost, rr, cc))

    return distances[(h * 5) - 1][(w * 5) - 1]
    
solve_2(test_input)

315

In [56]:
f"Solution 2: {solve_2(final_input)}"

'Solution 2: 2976'